In [ ]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
model = "gpt-4o-mini"

In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
user_statement = "White sand is made from fish poop https://oceanservice.noaa.gov/facts/sand.html"


# 2
SYSTEM_PROMPT = (
  "You are a fact checker. Verify the validity of the sentence provided by the user, given a reference. You must return a response in JSON format:"
  "{'isFactTrue': <true or false>, 'explanation': <explanation> }"
)
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_statement}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    response_format={ "type": "json_object" }
)
print(response.choices[0].message.content)

In [ ]:
def get_text_from_url(url):
    if url == "https://oceanservice.noaa.gov/facts/sand.html":
      return '''The famous white-sand beaches of Hawaii, for example, actually come from the poop of parrotfish. The fish bite and scrape algae off of rocks and dead corals with their parrot-like beaks, grind up the inedible calcium-carbonate reef material (made mostly of coral skeletons) in their guts, and then excrete it as sand.'''
tools = [
    {
        # 1
        "type": "function",
        # 2
        "function": {
            # 3
            "name": "get_text_from_url",
            # 4
            "description": "Get the text contents of a url",
            # 5
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url",
                    },
                },
                "required": ["url"],
            },
        },
    }
]
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  tools=tools,
  response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_text_from_url",
            "description": "Get the text contents of a url",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url",
                    },
                },
                "required": ["url"],
            },
        },
    }
]

response = client.chat.completions.create(
  model=model,
  messages=messages,
  tools=tools,
  response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

In [ ]:
print(response)

In [ ]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(tool_calls)

In [ ]:
import json

if tool_calls:
    # 2
    available_functions = {
        "get_text_from_url": get_text_from_url,
    }
    # 3
    messages.append(response_message) 
    # 4
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            url=function_args.get("url"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        ) 
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        response_format={"type": "json_object"},
    ) 

print(second_response.choices[0].message.content)